In [ ]:
!pip install ortools
!pip install dtale

In [10]:
import pandas as pd
import numpy as np
from ortools.sat.python import cp_model
from dtale import show
import dtale.app as dtale_app
dtale_app.USE_COLAB = True

In [8]:
# Clone the entire repo.
!git clone -l -s https://github.com/mswastik/optimization.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 14 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), done.
/content/cloned-repo
 CP.ipynb  'item master.xlsx'   orders.xlsx   README.md


In [ ]:
df=pd.read_excel('orders.xlsx')

In [9]:
df = df[df['Line']=='Toothpaste Line']
del df['Line']
df['due'] = df['Dispatch date'] - df['Dispatch date'].min()
df['due'] = df['due'].dt.days
df.reset_index(inplace=True,drop=True)

In [13]:
show(df,ignore_duplicate=True)

https://pi6pg94wws-496ff2e9c6d22116-40000-colab.googleusercontent.com/dtale/main/2